In [1]:
import re
import random
import numpy as np
from selenium import webdriver
from urllib.request import urlopen
from selenium.webdriver.common.by import By
from sklearn.cluster import SpectralClustering

In [2]:
random.seed(42)

In [3]:
a = []
a.append([])
a[0].append(1)
a.append([])
a.append([])
a.append([])
a[1].append(4)
print(a)

[[1], [4], [], []]


In [8]:
adj_list = []
edge_len = []
receptors = []
transcription_factors = []

similarity_matrix = np.zeros((6394, 6394))

proteins = {}
protein_list = []
curr_node = 0

In [24]:
fin = open("./datasets/4932.protein.links.v11.5.txt")
for line in fin:
    temp_list = line.split(" ")
    protein1 = temp_list[0].split(".")[1]
    protein2 = temp_list[1].split(".")[1]
    
    if(protein1 not in proteins.keys()):
        protein_list.append(protein1)
        proteins[protein1] = curr_node
        curr_node += 1
        adj_list.append([])
        edge_len.append([])
    
    if(protein2 not in proteins.keys()):
        protein_list.append(protein2)
        proteins[protein2] = curr_node
        curr_node += 1
        adj_list.append([])
        edge_len.append([])
    
    adj_list[proteins[protein1]].append(proteins[protein2])
    adj_list[proteins[protein2]].append(proteins[protein1])
    
    edge_len[proteins[protein1]].append(proteins[protein2])
    edge_len[proteins[protein2]].append(proteins[protein1])
fin.close()
    

In [ ]:
driver = webdriver.Chrome(executable_path="D:\chromedriver\chromedriver.exe")
temp_set = set(protein_list)
fout = open('./similarity.txt', 'a')

for protein in protein_list[6200:]:
    protein_index1 = proteins[protein]
    print(protein_index1)
    
    driver.get(f"https://spell.yeastgenome.org/search/show_results?search_string={protein}")
    driver.get("https://spell.yeastgenome.org/search/gene_results_as_text")

    try:
        # driver.find_element(By.XPATH, "//body/h1")
        raw_info = driver.find_element(By.XPATH, "//body/pre").text
    except:
        if(protein_index1 > 0):
            fout.write('\n')
        fout.write(f"{similarity_matrix[protein_index1, 0]}")
        for value in similarity_matrix[protein_index1, 1:] :
            fout.write(f",{value}")
        continue

    info = raw_info.split(" ")
    
    i = 17
    while(i < len(info)):
        protein2 = info[i]
        if(protein2 not in temp_set):
            i += 3
            continue
        protein_index2 = proteins[protein2]
        score = info[i+2].split("\n")[0]
        similarity_matrix[protein_index1][protein_index2] = float(score)
        i += 3
    
    if(protein_index1 > 0):
        fout.write('\n')
    fout.write(f"{similarity_matrix[protein_index1, 0]}")
    for value in similarity_matrix[protein_index1, 1:] :
        fout.write(f",{value}")
    
fout.close()
driver.quit()  

In [9]:
fin = open("../similarity.txt")
i = 0
for line in fin:
    temp_list = line.split(",")
    for j in range(len(temp_list)):
        similarity_matrix[i,j] = float(temp_list[j])
    i += 1
fin.close()

In [19]:
# print(proteins)
# print(proteins)
# print(len(adj_list))

In [20]:
fout = open('./datasets/protein_list.txt','w')
for protein in proteins.keys():
    fout.write(f"{protein}\n")
fout.close()

In [41]:
num_v = len(proteins)
k = 8
v = np.arange(num_v)
v_colors = [random.randint(0,k-1) for i in v]

In [ ]:
# x = 0
for protein in proteins.keys():
#     x += 1
#     if x%5 == 0 :
#         print(x)
    url = 'http://www.yeastract.com/view.php?existing=protein&proteinname=' + protein
    html_content = str(urlopen(url).read())
    matches = re.findall('Transcription&nbsp;Factor', html_content)
    if len(matches) > 0:
        transcription_factors.append(protein)
        # print(protein, 'T')
    matches = re.findall('receptor', html_content)
    if len(matches) > 0:
        receptors.append(protein)
        # print(protein, 'R')

# print(transcription_factors)

In [16]:
f = open("transcriptors.txt", "w")
for t in transcription_factors:
    f.write(t + "\n")
f.close()

f = open("receptors.txt", "w")
for r in receptors:
    f.write(r + "\n")
f.close()

In [26]:
fin = open("transcriptors.txt")
for protein in fin:
    transcription_factors.append(proteins[protein.split("\n")[0]])
fin.close()

fin = open("receptors.txt")
for protein in fin:
    receptors.append(proteins[protein.split("\n")[0]])
fin.close()

In [61]:
def dfs(transcription_factors, adj_list, edge_probs, v_colors, visited_colors, next_in_path, num_visited_colors, u, k, max_length, curr_length):
    visited_colors[v_colors[u]] = 1
    num_visited_colors += 1
    if(num_visited_colors == k and u in transcription_factors and curr_length > max_length[0]):
        max_length[0] = curr_length
        return True
    
    flag = False

    j = -1
    for v in adj_list[u]:
        j += 1
        if(visited_colors[v_colors[v]] == 1):
            continue
        flag = dfs(transcription_factors, adj_list, edge_probs, v_colors, visited_colors, next_in_path, num_visited_colors, v, k, max_length, curr_length + edge_probs[u][j])
        if(flag):
            next_in_path[u] = v
    
    visited_colors[v_colors[u]] = 0
    num_visited_colors -= 1

    return flag 

In [62]:
def get_best_pathway(receptors, transcription_factors, adj_list, edge_probs, v_colors, k):
    curr_max = 0
    curr_start_vertex = 0
    num_vertices = len(v_colors)
    next_in_path = [0] * num_vertices
    for u in receptors:
        max_length = [0]
        visited_colors = [0] * num_vertices
        dfs(transcription_factors, adj_list, edge_probs, v_colors, visited_colors, next_in_path, 0, u, k, max_length, 0)
        if(max_length[0] > curr_max):
            curr_max = max_length[0]
            curr_start_vertex = u
    
    best_path = []
    temp_node = curr_start_vertex
    for i in range(k):
        best_path.append(temp_node)
        temp_node = next_in_path[temp_node]
    
    return best_path, curr_max

In [63]:
best_path, max_score = get_best_pathway(receptors, transcription_factors, adj_list, edge_len, v_colors, k)

In [65]:
print(max_score, best_path)

9588 [5026, 4983, 4605]
